In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers.legacy import Adam
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.utils import compute_sample_weight

In [ ]:
data = pd.read_csv('./SalarayDataSet/ds_salaries.csv')
data.head()

# Data Modification

In [ ]:
data.isnull().sum()
cols_to_normalize = ['salary', 'salary_in_usd']
data[cols_to_normalize] = data[cols_to_normalize] / 100000

## Data Preprocessing and Splitting

In this section, we preprocess the dataset and split it into training and validation sets. The following steps are performed:

1. **Feature-Target Separation:** We separate the feature matrix `data_X` from the target variable `y` in the dataset.

    ```python
    data_X = data.drop('salary_in_usd', axis=1)
    y = data['salary_in_usd']
    ```

2. **Numeric and Categorical Feature Processing:** We identify numeric and categorical features in the dataset and apply appropriate preprocessing transformations.

    ```python
    numeric_features = data_X.select_dtypes(include=[np.number])
    numeric_transformer = StandardScaler()

    categorical_features = data_X.select_dtypes(include=['object'])
    categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features.columns),
            ('cat', categorical_transformer, categorical_features.columns)
        ])
    ```

3. **Dataset Splitting:** We split the dataset into training and validation sets using a specified test size and random state.

    ```python
    indices = data.index
    train_indices, valid_indices = train_test_split(indices, test_size=0.4, random_state=42)
    ```

4. **Applying Transformers:** We apply the preprocessing transformers to the training and validation feature sets.

    ```python
    x_train, y_train = data_X.loc[train_indices, :], y.loc[train_indices]
    x_valid, y_valid = data_X.loc[valid_indices, :], y.loc[valid_indices]

    X_train_preprocessed = preprocessor.fit_transform(x_train)
    X_valid_preprocessed = preprocessor.transform(x_valid)
    ```

These steps ensure that our dataset is properly preprocessed and ready for training and validation of machine learning models.

In [ ]:
data_X = data.drop('salary_in_usd', axis=1)
y = data['salary_in_usd']

numeric_features = data_X.select_dtypes(include=[np.number])
numeric_transformer = StandardScaler()

categorical_features = data_X.select_dtypes(include=['object'])
categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features.columns),
        ('cat', categorical_transformer, categorical_features.columns)
    ])
indices = data.index

train_indices, valid_indices = train_test_split(indices, test_size=0.4, random_state=42)
x_train, y_train = data_X.loc[train_indices, :], y.loc[train_indices]
x_valid, y_valid = data_X.loc[valid_indices, :], y.loc[valid_indices]

X_train_preprocessed = preprocessor.fit_transform(x_train)
X_valid_preprocessed = preprocessor.transform(x_valid)

## Early Stopping Configuration
We configure early stopping to monitor the validation loss and terminate training if the loss does not improve after a specified number of epochs.
```python
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
```

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Models

## Linear Regression Modeling with and without Early Stopping

In this section, we implement linear regression models with early stopping to prevent overfitting and improve generalization performance. The following steps are executed:

1. **Building Linear Regression Model:** Two linear regression models are constructed using the Keras Sequential API. The first model (`linear_regression_early_stopping`) is equipped with early stopping, while the second model (`linear_regression`) is trained without early stopping.

    ```python
    linear_regression_early_stopping = Sequential([
        Dense(1, input_shape=(X_train_preprocessed.shape[1],), activation='linear')
    ])

    linear_regression_early_stopping.compile(optimizer=Adam(learning_rate=0.001),
                                              loss='mean_squared_error',
                                              metrics=['mae'])

    linear_regression = Sequential([
        Dense(1, input_shape=(X_train_preprocessed.shape[1],), activation='linear')
    ])

    linear_regression.compile(optimizer=Adam(learning_rate=0.001),
                               loss='mean_squared_error',
                               metrics=['mae'])
    ```

2. **Model Training:** Both models are trained on the preprocessed training data (`X_train_preprocessed` and `y_train`) with the validation data (`X_valid_preprocessed` and `y_valid`) used for validation during training. The `callbacks` parameter is set to include early stopping for the first model.

    ```python
    linear_history_early_stopping = linear_regression_early_stopping.fit(X_train_preprocessed, y_train,
                                                                          validation_data=(X_valid_preprocessed, y_valid),
                                                                          epochs=400,
                                                                          batch_size=32,
                                                                          verbose=2,
                                                                          callbacks=[early_stopping])

    linear_history = linear_regression.fit(X_train_preprocessed, y_train,
                                           validation_data=(X_valid_preprocessed, y_valid),
                                           epochs=400,
                                           batch_size=32,
                                           verbose=2)
    ```

3. **Evaluation:** After training, both models are evaluated on the validation set to assess their performance using mean squared error (MSE) and mean absolute error (MAE) metrics.

    ```python
    linear_evaluation_result_early_stopping = linear_regression_early_stopping.evaluate(X_valid_preprocessed, y_valid, verbose=0)

    linear_evaluation_result = linear_regression.evaluate(X_valid_preprocessed, y_valid, verbose=0)
    ```

These steps facilitate the training and evaluation of linear regression models, allowing us to compare their performance with and without early stopping.


In [ ]:
linear_regression_early_stopping = Sequential([
    Dense(1, input_shape=(X_train_preprocessed.shape[1],), activation='linear')
])

linear_regression_early_stopping.compile(optimizer=Adam(learning_rate=0.001),  
              loss='mean_squared_error', 
              metrics=['mae']) 

linear_history_early_stopping = linear_regression_early_stopping.fit(X_train_preprocessed, y_train,
                    validation_data=(X_valid_preprocessed, y_valid),
                    epochs=400,
                    batch_size=32,
                    verbose=2,
                    callbacks=[early_stopping])

linear_evaluation_result_early_stopping = linear_regression_early_stopping.evaluate(X_valid_preprocessed, y_valid, verbose=0)

In [ ]:
linear_regression = Sequential([
    Dense(1, input_shape=(X_train_preprocessed.shape[1],), activation='linear')
])

linear_regression.compile(optimizer=Adam(learning_rate=0.001),  
              loss='mean_squared_error',  
              metrics=['mae']) 

linear_history = linear_regression.fit(X_train_preprocessed, y_train,
                    validation_data=(X_valid_preprocessed, y_valid),
                    epochs=400,
                    batch_size=32,
                    verbose=2)

linear_evaluation_result = linear_regression.evaluate(X_valid_preprocessed, y_valid, verbose=0)

### Results

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Plot for linear_history_early_stopping
axs[0].plot(linear_history_early_stopping.history['loss'], label='Training Loss')
axs[0].plot(linear_history_early_stopping.history['val_loss'], label='Validation Loss')
axs[0].set_title('Early Stopping')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].grid(True)
axs[0].legend()

# Plot for linear_history
axs[1].plot(linear_history.history['loss'], label='Training Loss')
axs[1].plot(linear_history.history['val_loss'], label='Validation Loss')
axs[1].set_title('Without Early Stopping')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Loss')
axs[1].grid(True)
axs[1].legend()

plt.subplots_adjust(top=0.2, bottom=0.0)
plt.show()

## Multi-Layer Perceptron (MLP) Modeling with Early Stopping

This section outlines the process of building, training, and evaluating Multi-Layer Perceptron (MLP) models with early stopping applied to prevent overfitting. The following steps are involved:

1. **Model Architecture Definition:** Two MLP models are defined using the Keras Sequential API. Each model consists of multiple dense layers with rectified linear unit (ReLU) activation functions, batch normalization for improving convergence, and dropout layers for regularization.

    ```python
    mlp_model_early_stopping = Sequential([
        Dense(128, activation='relu', input_dim=X_train_preprocessed.shape[1]),
        BatchNormalization(),
    
        Dense(256, activation='relu'),
        Dropout(0.3),  

        Dense(128, activation='relu'),
        Dropout(0.3), 

        Dense(1, activation='linear')
    ])
    ```

2. **Model Compilation:** Both MLP models are compiled using the Adam optimizer, mean squared error (MSE) as the loss function, and mean absolute error (MAE) as the metric to monitor during training.

    ```python
    mlp_model_early_stopping.compile(optimizer=Adam(learning_rate=0.001),
                                      loss='mean_squared_error',
                                      metrics=['mae'])
    ```

3. **Model Training:** The MLP models are trained on the preprocessed training data (`X_train_preprocessed` and `y_train`) with early stopping applied to one of the models. 

    ```python
    mlp_history_early_stopping = mlp_model_early_stopping.fit(X_train_preprocessed, y_train,
                                                              validation_data=(X_valid_preprocessed, y_valid),
                                                              epochs=400,
                                                              batch_size=32,
                                                              verbose=2,
                                                              callbacks=[early_stopping])
    ```

4. **Evaluation:** After training, the performance of both models is evaluated on the validation set to assess their predictive capabilities using MSE and MAE metrics.

    ```python
    mlp_evaluation_result_early_stopping = mlp_model_early_stopping.evaluate(X_valid_preprocessed, y_valid, verbose=0)
    ```

These steps demonstrate the construction and training of MLP models with early stopping, allowing for better generalization and improved performance on unseen data.

In [ ]:
mlp_model_early_stopping = Sequential([
    Dense(128, activation='relu', input_dim=X_train_preprocessed.shape[1]),
    BatchNormalization(),

    Dense(256, activation='relu'),
    Dropout(0.3),  

    Dense(128, activation='relu'),
    Dropout(0.3),  

    Dense(1, activation='linear')
])

mlp_model_early_stopping.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mae'])

mlp_history_early_stopping = mlp_model_early_stopping.fit(X_train_preprocessed, y_train,
                    validation_data=(X_valid_preprocessed, y_valid),
                    epochs=400,  
                    batch_size=32,  
                    verbose=2,
                    callbacks=[early_stopping])

mlp_evaluation_result_early_stopping = mlp_model_early_stopping.evaluate(X_valid_preprocessed, y_valid, verbose=0)

In [ ]:
mlp_model = Sequential([
    Dense(128, activation='relu', input_dim=X_train_preprocessed.shape[1]),
    BatchNormalization(),

    Dense(256, activation='relu'),
    Dropout(0.3),  

    Dense(128, activation='relu'),
    Dropout(0.3), 

    Dense(1, activation='linear')
])

mlp_model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mae'])


mlp_history = mlp_model.fit(X_train_preprocessed, y_train,
                    validation_data=(X_valid_preprocessed, y_valid),
                    epochs=400,  
                    batch_size=32, 
                    verbose=2)

mlp_evaluation_result = mlp_model.evaluate(X_valid_preprocessed, y_valid, verbose=0)

### Results

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Plot for mlp_history_early_stopping
axs[0].plot(mlp_history_early_stopping.history['loss'], label='Training Loss')
axs[0].plot(mlp_history_early_stopping.history['val_loss'], label='Validation Loss')
axs[0].set_title('Early Stopping')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].grid(True)
axs[0].legend()

# Plot for mlp_history
axs[1].plot(mlp_history.history['loss'], label='Training Loss')
axs[1].plot(mlp_history.history['val_loss'], label='Validation Loss')
axs[1].set_title('Without Early Stopping')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Loss')
axs[1].grid(True)
axs[1].legend()

plt.subplots_adjust(top=0.2, bottom=0.0)
plt.show()

## Multi-Layer Perceptron (MLP) Modeling with Sample Weights

This section describes the implementation of sample weights in Multi-Layer Perceptron (MLP) models to address class imbalance issues. The steps involved in incorporating sample weights into the MLP modeling process are outlined below:

1. **Computing Sample Weights:** Sample weights are computed using the `compute_sample_weight` function from scikit-learn, with the 'balanced' strategy applied to ensure balanced class weights.

    ```python
    sample_weights_train = compute_sample_weight('balanced', y_train)
    ```

2. **Model Architecture Definition:** Two MLP models are defined using the Keras Sequential API. Each model comprises multiple dense layers with ReLU activation functions, batch normalization, and dropout layers for regularization.

    ```python
    mlp_weight_model_early_stopping = Sequential([
        Dense(128, activation='relu', input_dim=X_train_preprocessed.shape[1]),
        BatchNormalization(),

        Dense(256, activation='relu'),
        Dropout(0.3), 

        Dense(128, activation='relu'),
        Dropout(0.3),  

        Dense(1, activation='linear')
    ])
    ```

3. **Model Compilation:** Both MLP models are compiled using the Adam optimizer, mean squared error (MSE) as the loss function, mean absolute error (MAE) as the metric, and 'temporal' sample weight mode to incorporate sample weights during training.

    ```python
    mlp_weight_model_early_stopping.compile(optimizer=Adam(learning_rate=0.001),
                                             loss='mean_squared_error',
                                             metrics=['mae'],
                                             sample_weight_mode='temporal')
    ```

4. **Model Training:** The MLP models are trained on the preprocessed training data (`X_train_preprocessed` and `y_train`) with early stopping applied to one of the models. Sample weights are passed as an argument to the `fit` method to incorporate them during training.

    ```python
    weight_model_history_early_stopping = mlp_weight_model_early_stopping.fit(X_train_preprocessed, y_train,
                                                                                validation_data=(X_valid_preprocessed, y_valid),
                                                                                epochs=400,
                                                                                batch_size=32,
                                                                                verbose=2,
                                                                                sample_weight=sample_weights_train,
                                                                                callbacks=[early_stopping])
    ```

5. **Evaluation:** After training, the performance of both models is evaluated on the validation set to assess their predictive capabilities using MSE and MAE metrics.

    ```python
    weight_model_evaluation_result_early_stopping = mlp_weight_model_early_stopping.evaluate(X_valid_preprocessed, y_valid, verbose=0)
    ```

These steps demonstrate the integration of sample weights into MLP modeling to address class imbalance issues and improve model performance.

In [ ]:
sample_weights_train = compute_sample_weight('balanced', y_train)
mlp_weight_model_early_stopping = Sequential([
    Dense(128, activation='relu', input_dim=X_train_preprocessed.shape[1]),
    BatchNormalization(),

    Dense(256, activation='relu'),
    Dropout(0.3), 

    Dense(128, activation='relu'),
    Dropout(0.3),  

    Dense(1, activation='linear')
])

mlp_weight_model_early_stopping.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error', 
              metrics=['mae'],
              sample_weight_mode='temporal') 

weight_model_history_early_stopping = mlp_weight_model_early_stopping.fit(X_train_preprocessed, y_train,
                    validation_data=(X_valid_preprocessed, y_valid),
                    epochs=400,
                    batch_size=32,
                    verbose=2,
                    sample_weight=sample_weights_train,
                    callbacks=[early_stopping])

weight_model_evaluation_result_early_stopping = mlp_weight_model_early_stopping.evaluate(X_valid_preprocessed, y_valid, verbose=0)

In [ ]:
mlp_weight_model = Sequential([
    Dense(128, activation='relu', input_dim=X_train_preprocessed.shape[1]),
    BatchNormalization(),

    Dense(256, activation='relu'),
    Dropout(0.3),  

    Dense(128, activation='relu'),
    Dropout(0.3),  

    Dense(1, activation='linear')
])

mlp_weight_model.compile(optimizer=Adam(learning_rate=0.001),
                         sample_weight_mode='temporal',
              loss='mean_squared_error', 
              metrics=['mae'])

weight_model_history = mlp_weight_model.fit(X_train_preprocessed, y_train,
                    validation_data=(X_valid_preprocessed, y_valid),
                    epochs=400,
                    batch_size=32,
                    sample_weight=sample_weights_train,
                    verbose=2)

weight_model_evaluation_result = mlp_weight_model.evaluate(X_valid_preprocessed, y_valid, verbose=0)

### Results

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Plot for weight_model_history_early_stopping
axs[0].plot(weight_model_history_early_stopping.history['loss'], label='Training Loss')
axs[0].plot(weight_model_history_early_stopping.history['val_loss'], label='Validation Loss')
axs[0].set_title('Early Stopping')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].grid(True)
axs[0].legend()

# Plot for weight_model_history
axs[1].plot(weight_model_history.history['loss'], label='Training Loss')
axs[1].plot(weight_model_history.history['val_loss'], label='Validation Loss')
axs[1].set_title('Without Early Stopping')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Loss')
axs[1].grid(True)
axs[1].legend()

plt.subplots_adjust(top=0.4, bottom=0.0)
plt.show()

# Conclusion

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot for models with early stopping
axs[0].plot(linear_history_early_stopping.history['loss'], label='Linear')
axs[0].plot(mlp_history_early_stopping.history['loss'], label='MLP')
axs[0].plot(weight_model_history_early_stopping.history['loss'], label='MLP with Weight')
axs[0].set_title('Models with Early Stopping')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].grid(True)
axs[0].legend()

# Plot for models without early stopping
axs[1].plot(linear_history.history['loss'], label='Linear')
axs[1].plot(mlp_history.history['loss'], label='MLP')
axs[1].plot(weight_model_history.history['loss'], label='MLP with Weight')
axs[1].set_title('Models without Early Stopping')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Loss')
axs[1].grid(True)
axs[1].legend()

# Plot for all models
axs[2].plot(linear_history_early_stopping.history['loss'], label='Linear (Early Stopping)')
axs[2].plot(mlp_history_early_stopping.history['loss'], label='MLP (Early Stopping)')
axs[2].plot(weight_model_history_early_stopping.history['loss'], label='MLP with Weight (Early Stopping)')
axs[2].plot(linear_history.history['loss'], label='Linear')
axs[2].plot(mlp_history.history['loss'], label='MLP')
axs[2].plot(weight_model_history.history['loss'], label='MLP with Weight')
axs[2].set_title('All Models')
axs[2].set_xlabel('Epochs')
axs[2].set_ylabel('Loss')
axs[2].grid(True)
axs[2].legend()

plt.subplots_adjust(top=0.4, bottom=0.0)
plt.show()


## Convolutional Neural Network (CNN) Modeling with and without Early Stopping

This section outlines the implementation of Convolutional Neural Network (CNN) models with early stopping for regression tasks. The process involves the following steps:

1. **CNN Model Definition:** Two CNN models are defined using the Keras Sequential API. Each model consists of convolutional layers with ReLU activation functions, max-pooling layers for downsampling, and dense layers for regression.

    ```python
    cnn_model_early_stopping = Sequential([
        Conv1D(32, 3, activation='relu', input_shape=(248, 1)),
        MaxPooling1D(2),

        Conv1D(64, 3, activation='relu'),
        MaxPooling1D(2),

        Conv1D(128, 3, activation='relu'),
        MaxPooling1D(2),

        Flatten(),

        Dense(64, activation='relu'),

        Dense(1)
    ])
    ```

2. **Model Compilation:** Both CNN models are compiled using the Adam optimizer, mean squared error (MSE) as the loss function, and mean absolute error (MAE) as the metric.

    ```python
    cnn_model_early_stopping.compile(optimizer=Adam(learning_rate=0.001),
                      loss='mean_squared_error',
                      metrics=['mae'])
    ```

3. **Model Training:** The CNN models are trained on the preprocessed training data with early stopping applied to one of the models. The `fit` method is used to train the models with specified epochs, batch size, and verbose mode.

    ```python
    cnn_model_history_early_stopping = cnn_model_early_stopping.fit(X_train_preprocessed, y_train,
                        validation_data=(X_valid_preprocessed, y_valid),
                        epochs=400,
                        batch_size=32,
                        verbose=2,
                        callbacks=[early_stopping])
    ```

4. **Evaluation:** After training, the performance of both models is evaluated on the validation set using MSE and MAE metrics.

    ```python
    cnn_model_evalutaion_result_early_stopping = cnn_model_early_stopping.evaluate(X_valid_preprocessed, y_valid, verbose=0)
    ```

These steps illustrate the process of implementing CNN models with early stopping for regression tasks, providing insights into model architecture, compilation, training, and evaluation.

In [ ]:
train_indices, valid_indices = train_test_split(indices, test_size=0.3, random_state=42)
x_train, y_train = data_X.loc[train_indices, :], y.loc[train_indices]
x_valid, y_valid = data_X.loc[valid_indices, :], y.loc[valid_indices]

X_train_preprocessed = preprocessor.fit_transform(x_train)
X_valid_preprocessed = preprocessor.transform(x_valid)

cnn_model_early_stopping = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(248, 1)),
    MaxPooling1D(2),

    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),

    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),

    Flatten(),

    Dense(64, activation='relu'),

    Dense(1)
])

cnn_model_early_stopping.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error',
                  metrics=['mae'])

cnn_model_history_early_stopping = cnn_model_early_stopping.fit(X_train_preprocessed, y_train,
                    validation_data=(X_valid_preprocessed, y_valid),
                    epochs=400,
                    batch_size=32,
                    verbose=2,
                    callbacks=[early_stopping])

cnn_model_evalutaion_result_early_stopping = cnn_model_early_stopping.evaluate(X_valid_preprocessed, y_valid, verbose=0)

In [ ]:
cnn_model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(248, 1)),
    MaxPooling1D(2),

    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),

    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),

    Flatten(),

    Dense(64, activation='relu'),

    Dense(1)
])

cnn_model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error',
                  metrics=['mae'])

cnn_model.summary()

cnn_model_history = cnn_model.fit(X_train_preprocessed, y_train,
                    validation_data=(X_valid_preprocessed, y_valid),
                    epochs=400,
                    batch_size=32,
                    verbose=2)

cnn_model_evalutaion_result = cnn_model.evaluate(X_valid_preprocessed, y_valid, verbose=0)

## Results

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

axs[0].plot(cnn_model_history_early_stopping.history['loss'], label='Training Loss')
axs[0].plot(cnn_model_history_early_stopping.history['val_loss'], label='Validation Loss')
axs[0].set_title('Early Stopping')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].grid(True)
axs[0].legend()

axs[1].plot(cnn_model_history.history['loss'], label='Training Loss')
axs[1].plot(cnn_model_history.history['val_loss'], label='Validation Loss')
axs[1].set_title('Without Early Stopping')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Loss')
axs[1].grid(True)
axs[1].legend()

plt.subplots_adjust(top=0.1, bottom=0.0)
plt.show()

# Comparing All Models
## Training Loss Early Stopping

In [ ]:
print(f"Linear Regression model loss is : {linear_history_early_stopping.history['loss'][-1]}\nMLP model loss is: {mlp_history_early_stopping.history['loss'][-1]}\nMLP model with updated weights loss is : {weight_model_history_early_stopping.history['loss'][-1]}\nCNN model loss is: {cnn_model_history_early_stopping.history['loss'][-1]}\n")

In [ ]:
history_objects_early_stopping = [linear_history_early_stopping, mlp_history_early_stopping, 
                   weight_model_history_early_stopping, cnn_model_history_early_stopping]

model_names = ['Linear Regression', 'MLP', 'MLP with Updated Weights', 'CNN']

plt.figure(figsize=(10, 6))
for history, name in zip(history_objects_early_stopping, model_names):
    plt.plot(history.history['loss'], label=f'{name} Training Loss', linestyle='--')

plt.title('Training Loss Early Stopping')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## Validation Loss Early Stopping

In [ ]:
print(f"Linear Regression model validation loss is : {linear_history_early_stopping.history['val_loss'][-1]}\nMLP model validation loss is: {mlp_history_early_stopping.history['val_loss'][-1]}\nMLP model with updated weights validation loss is : {weight_model_history_early_stopping.history['val_loss'][-1]}\nCNN model validation loss is: {cnn_model_history_early_stopping.history['val_loss'][-1]}")

In [ ]:
plt.figure(figsize=(10, 6))
for history, name in zip(history_objects_early_stopping, model_names):
    plt.plot(history.history['val_loss'], label=f'{name} Validation Loss', linestyle='--')

plt.title('Validation Loss Early Stopping')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## Training Loss Without Early Stopping

In [ ]:
print(f"Linear Regression model loss is : {linear_history.history['loss'][-1]}\nMLP model loss is: {mlp_history.history['loss'][-1]}\nMLP model with updated weights loss is : {weight_model_history.history['loss'][-1]}\nCNN model loss is: {cnn_model_history.history['loss'][-1]}\n")

In [ ]:
history_objects = [linear_history, mlp_history, 
                   weight_model_history, cnn_model_history]

plt.figure(figsize=(10, 6))
for history, name in zip(history_objects, model_names):
    plt.plot(history.history['loss'], label=f'{name} Training Loss', linestyle='--')

plt.title('Training Loss Without Early Stopping')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## Validation Loss Without Early Stopping

In [ ]:
print(f"Linear Regression model validation loss is : {linear_history.history['val_loss'][-1]}\nMLP model validation loss is: {mlp_history.history['val_loss'][-1]}\nMLP model with updated weights validation loss is : {weight_model_history.history['val_loss'][-1]}\nCNN model validation loss is: {cnn_model_history.history['val_loss'][-1]}")

In [ ]:
plt.figure(figsize=(10, 6))
for history, name in zip(history_objects, model_names):
    plt.plot(history.history['val_loss'], label=f'{name} Validation Loss', linestyle='--')

plt.title('Validation Loss Early Stopping')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()